In [1]:
import os
import pickle
import pandas as pd
import numpy as np

##### Path to outputdict file

In [2]:
output_dict_path = "outputDict" #path to output dict

In [3]:
os.path.isfile(output_dict_path)

True

##### Load the file

In [4]:
output_dict = pickle.load(open(output_dict_path,"rb"))

In [5]:
output_dict

,Labels,Prob,ethnicity,gender,age
0,0,0.443426,WHITE,M,63
1,0,0.471146,WHITE,M,91
2,0,0.452288,WHITE,M,73
3,0,0.514837,WHITE,F,66
4,0,0.688411,UNKNOWN,M,78
...,...,...,...,...,...
2995,0,0.398999,WHITE,M,31
2996,0,0.515015,WHITE,M,63
2997,0,0.444717,WHITE,F,63
2998,0,0.465339,WHITE,M,86


##### Add predicted label using the probability

In [6]:
output_dict["Predicted"] = output_dict.apply(lambda row:row.Labels if row.Prob>=0.5 else 1-row.Labels, axis=1)

In [7]:
output_dict

,Labels,Prob,ethnicity,gender,age,Predicted
0,0,0.443426,WHITE,M,63,1
1,0,0.471146,WHITE,M,91,1
2,0,0.452288,WHITE,M,73,1
3,0,0.514837,WHITE,F,66,0
4,0,0.688411,UNKNOWN,M,78,0
...,...,...,...,...,...,...
2995,0,0.398999,WHITE,M,31,1
2996,0,0.515015,WHITE,M,63,0
2997,0,0.444717,WHITE,F,63,1
2998,0,0.465339,WHITE,M,86,1


##### Bin age

In [8]:
output_dict["age_binned"] = output_dict.age.apply(lambda x:"{}-{}".format((x//10)*10,(x//10 + 1)*10))

##### List of sensitive columns

In [9]:
sensitive_columns = ["ethnicity", "gender", "age_binned"]

##### Function to compute the Confusion Matrix parameters

In [34]:
def get_cm_parameters(gt, pred):
    zipped_gt_pred = list(zip(gt,pred))
    tp = len([pair for pair in zipped_gt_pred if pair == (1,1)])
    tn = len([pair for pair in zipped_gt_pred if pair == (0,0)])
    fp = len([pair for pair in zipped_gt_pred if pair == (0,1)])
    fn = len([pair for pair in zipped_gt_pred if pair == (1,0)])
    
    try:
        tpr = tp/(tp + fn)
    except ZeroDivisionError:
        tpr = None
    try:
        tnr = tn/(tn + fp)
    except ZeroDivisionError:
        tnr = None
    try:
        fpr = fp/(fp + tn)
    except ZeroDivisionError:
        fpr = None
    try:
        fnr = fn/(fn + tp)
    except ZeroDivisionError:
        fnr = None
    try:
        acc = (tp+tn)/(len(zipped_gt_pred))
    except ZeroDivisionError:
        acc = None
    
    return tp, tn, fp, fn, tpr, tnr, fpr, fnr, acc

##### Generate fairness report

In [35]:
report_list = []
for sens_col in sensitive_columns:
    for group, aggregate in output_dict.groupby(sens_col):
        tmp_dct = {"sensitive_attribute": sens_col}
        tp, tn, fp, fn, tpr, tnr, fpr, fnr, acc = get_cm_parameters(list(aggregate.Labels), list(aggregate.Predicted))
        tmp_dct.update(dict(
            group=group,tp=tp, tn=tn, fp=fp, fn=fn, tpr=tpr, tnr=tnr, fpr= fpr, fnr=fnr, accuracy=acc    
            )
        )
        report_list.append(tmp_dct)

In [36]:
report = pd.DataFrame(report_list)
report_groups = {c:i for i,c in enumerate(report.sensitive_attribute.unique())}

In [37]:
def highlight(s):
    colors = [['background-color: yellow'], ['background-color: green'], ['background-color: red']]
    return colors[report_groups[s.sensitive_attribute]%len(colors)] * len(s)

In [38]:
try:
    import jinja2
    display(report.style.apply(highlight, axis=1))
except ImportError:
    display(report)

,sensitive_attribute,group,tp,tn,fp,fn,tpr,tnr,fpr,fnr,accuracy
0,ethnicity,AMERICAN INDIAN/ALASKA NATIVE,0,3,5,0,nan,0.375000,0.625000,nan,0.375000
1,ethnicity,ASIAN,5,42,31,2,0.714286,0.575342,0.424658,0.285714,0.587500
2,ethnicity,BLACK/AFRICAN AMERICAN,13,131,157,13,0.500000,0.454861,0.545139,0.500000,0.458599
3,ethnicity,HISPANIC/LATINO,3,39,52,6,0.333333,0.428571,0.571429,0.666667,0.420000
4,ethnicity,OTHER,9,43,75,6,0.600000,0.364407,0.635593,0.400000,0.390977
5,ethnicity,UNABLE TO OBTAIN,5,15,10,5,0.500000,0.600000,0.400000,0.500000,0.571429
6,ethnicity,UNKNOWN,34,106,128,29,0.539683,0.452991,0.547009,0.460317,0.471380
7,ethnicity,WHITE,63,775,1101,94,0.401274,0.413113,0.586887,0.598726,0.412199
8,gender,F,66,550,647,60,0.523810,0.459482,0.540518,0.476190,0.465608
9,gender,M,66,604,912,95,0.409938,0.398417,0.601583,0.590062,0.399523
